# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [70]:
import numpy as np 
import pandas as pd 
import requests
import math
import scipy 
from scipy import stats 
from scipy.stats import percentileofscore as score
import xlsxwriter 


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [7]:
stocks = pd.read_csv("sp_500_stocks.csv")
display(stocks.head())
print(stocks.shape)

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


(505, 1)


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [8]:
from secrets import IEX_CLOUD_API_TOKEN
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url,).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2209414194137,
 'week52high': 140.12,
 'week52low': 56.19,
 'week52change': 0.8648657488513024,
 'sharesOutstanding': 17448549676,
 'float': 0,
 'avg10Volume': 120546007,
 'avg30Volume': 107086159,
 'day200MovingAvg': 110.42,
 'day50MovingAvg': 122.54,
 'employees': 137618,
 'ttmEPS': 3.44,
 'ttmDividendRate': 0.8113781902807161,
 'dividendYield': 0.006377349656421145,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-23',
 'nextEarningsDate': '0',
 'peRatio': 38.8404693937996,
 'beta': 0,
 'maxChangePercent': 48.83850469125508,
 'year5ChangePercent': 4.344792966990817,
 'year2ChangePercent': 2.263593705914048,
 'year1ChangePercent': 0.8540386653897125,
 'ytdChangePercent': 0.767634008104645,
 'month6ChangePercent': 0.4651021496722006,
 'month3ChangePercent': 0.19533232137964351,
 'month1ChangePercent': 0.0687525277614192,
 'day30ChangePercent': 0.07034108372576976,
 'day5ChangePercent': 0.03635402646981385}

In [9]:
data['year1ChangePercent']

0.8540386653897125

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
#Converts list to sublists.
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = [] 
for i in range(0, len(symbol_groups)):
    #print(i) #Prints the indices of all the symbol groups ie. sublists
    #print(symbol_groups[i]) #Prints the actual sublists. 
    
    symbol_strings.append(','.join(symbol_groups[i])) #joining the ticers symbols
    
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    #print(symbol_string)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    #print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    #print(data)
    
    for symbol in symbol_string.split(","):
        print(symbol)
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
            
        ], index=my_columns),
        ignore_index=True
        )


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [35]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = [] 
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

for symbol_string in symbol_strings:
    print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [36]:
final_dataframe = pd.DataFrame( columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(","):
        
        final_dataframe = final_dataframe.append( 
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'

            ], index=my_columns),
            ignore_index=True)
        

In [37]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,121.600,0.438539,N/A
1,AAL,16.640,-0.427707,N/A
2,AAP,163.700,0.0318164,N/A
3,AAPL,126.994,0.833264,N/A
4,ABBV,107.630,0.247655,N/A
...,...,...,...,...
500,YUM,112.700,0.12375,N/A
501,ZBH,152.930,0.0292425,N/A
502,ZBRA,378.000,0.474765,N/A
503,ZION,41.730,-0.177451,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [42]:
final_dataframe.sort_values('One-Year Price Return', ascending= False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

<ipython-input-42-8a25f70e4a30>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe.sort_values('One-Year Price Return', ascending= False, inplace=True)


,level_0,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,0,78,CARR,38.840,2.3133,N/A
1,1,31,AMD,96.620,1.29261,N/A
2,2,345,NVDA,543.730,1.29007,N/A
3,3,387,PYPL,242.500,1.1842,N/A
4,4,275,LB,39.270,1.16538,N/A
5,5,23,ALB,139.100,1.0497,N/A
6,6,339,NOW,588.320,1.04072,N/A
7,7,24,ALGN,543.910,0.977878,N/A
8,8,236,IDXX,503.850,0.921285,N/A
9,9,85,CDNS,138.190,0.920632,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [44]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio:")
    
    try:
        float(portfolio_size)
    except:
        print("That is not a number!\nTry again.")
        portfolio_size = input("Enter the size of your portfolio:")
        
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:12000000
12000000


In [48]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = position_size

final_dataframe    

,level_0,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,0,78,CARR,38.840,2.3133,240000
1,1,31,AMD,96.620,1.29261,240000
2,2,345,NVDA,543.730,1.29007,240000
3,3,387,PYPL,242.500,1.1842,240000
4,4,275,LB,39.270,1.16538,240000
5,5,23,ALB,139.100,1.0497,240000
6,6,339,NOW,588.320,1.04072,240000
7,7,24,ALGN,543.910,0.977878,240000
8,8,236,IDXX,503.850,0.921285,240000
9,9,85,CDNS,138.190,0.920632,240000


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [138]:
hqm_columns = [
    "Ticker",
    "Price",
    "Number of Shares to Buy",
    "One-Year Price Return",
    "One-Year Return Percentile",
    "Six-Month Price Return",
    "Six-Month Return Percentile",
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
for symbol_string in symbol_strings :
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(","):
        hqm_dataframe = hqm_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            "N/A",
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            "N/A",
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
        ], index=hqm_columns),
        ignore_index=True)
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,120.60,N/A,0.433196,N/A,0.364018,N/A,0.203362,N/A,0.100612,N/A,N/A
1,AAL,16.72,N/A,-0.423095,N/A,0.0320571,N/A,0.252105,N/A,0.303795,N/A,N/A
2,AAP,163.72,N/A,0.031878,N/A,0.111535,N/A,0.0422614,N/A,0.0976152,N/A,N/A
3,AAPL,132.64,N/A,0.862733,N/A,0.464289,N/A,0.196875,N/A,0.0686236,N/A,N/A
4,ABBV,106.74,N/A,0.24709,N/A,0.109801,N/A,0.176302,N/A,0.0500869,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,113.84,N/A,0.126033,N/A,0.263759,N/A,0.20185,N/A,0.0494894,N/A,N/A
501,ZBH,154.25,N/A,0.0287096,N/A,0.216694,N/A,0.0976999,N/A,0.0364429,N/A,N/A
502,ZBRA,375.00,N/A,0.491999,N/A,0.424755,N/A,0.466845,N/A,0.0415623,N/A,N/A
503,ZION,41.39,N/A,-0.173497,N/A,0.167468,N/A,0.328826,N/A,0.0710421,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [139]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]
time_periods

for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row,percentile_col] = 0
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0

hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,120.60,N/A,0.433196,0,0.364018,0,0.203362,0,0.100612,0,N/A
1,AAL,16.72,N/A,-0.423095,0,0.0320571,0,0.252105,0,0.303795,0,N/A
2,AAP,163.72,N/A,0.031878,0,0.111535,0,0.0422614,0,0.0976152,0,N/A
3,AAPL,132.64,N/A,0.862733,0,0.464289,0,0.196875,0,0.0686236,0,N/A
4,ABBV,106.74,N/A,0.24709,0,0.109801,0,0.176302,0,0.0500869,0,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,113.84,N/A,0.126033,0,0.263759,0,0.20185,0,0.0494894,0,N/A
501,ZBH,154.25,N/A,0.0287096,0,0.216694,0,0.0976999,0,0.0364429,0,N/A
502,ZBRA,375.00,N/A,0.491999,0,0.424755,0,0.466845,0,0.0415623,0,N/A
503,ZION,41.39,N/A,-0.173497,0,0.167468,0,0.328826,0,0.0710421,0,N/A


In [154]:
#hqm_dataframe.fillna(0, inplace=True)

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row,percentile_col] = 0
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col],hqm_dataframe.loc[row, change_col])/100
        
        
hqm_dataframe

c:\users\i2989\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALGN,537.33,37,0.970887,0.92,1.06399,0.98,0.65407,1,0.208774,0.86,98.5149
1,FCX,25.42,786,0.941768,0.9,1.38907,1,0.45281,0.74,0.168387,0.74,96.3861
2,TWTR,58.25,343,0.770259,0.8,0.701209,0.84,0.411013,0.68,0.282054,0.98,96.2871
3,ALB,141.50,141,1.04082,0.96,0.803159,0.92,0.421946,0.7,0.11116,0.5,93.9109
4,XLNX,154.01,129,0.599436,0.64,0.599741,0.74,0.503034,0.88,0.122708,0.56,93.0198
5,PYPL,237.63,84,1.20797,0.98,0.43938,0.32,0.345681,0.44,0.242143,0.94,92.5248
6,LRCX,486.89,41,0.683949,0.7,0.559557,0.72,0.599451,0.94,0.101799,0.42,92.1782
7,AMD,96.86,206,1.25566,1,0.785607,0.9,0.284955,0.2,0.127235,0.64,91.8812
8,CDNS,135.32,147,0.929555,0.88,0.455851,0.4,0.336369,0.4,0.174304,0.78,91.3861
9,PAYC,442.64,45,0.705389,0.74,0.401371,0.2,0.617612,0.98,0.124278,0.62,90.8911


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [157]:
from statistics import mean  

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
        
hqm_dataframe
    

c:\users\i2989\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALGN,537.33,37,0.970887,0.92,1.06399,0.98,0.65407,1,0.208774,0.86,0.94
1,FCX,25.42,786,0.941768,0.9,1.38907,1,0.45281,0.74,0.168387,0.74,0.845
2,TWTR,58.25,343,0.770259,0.8,0.701209,0.84,0.411013,0.68,0.282054,0.98,0.825
3,ALB,141.50,141,1.04082,0.96,0.803159,0.92,0.421946,0.7,0.11116,0.5,0.77
4,XLNX,154.01,129,0.599436,0.64,0.599741,0.74,0.503034,0.88,0.122708,0.56,0.705
5,PYPL,237.63,84,1.20797,0.98,0.43938,0.32,0.345681,0.44,0.242143,0.94,0.67
6,LRCX,486.89,41,0.683949,0.7,0.559557,0.72,0.599451,0.94,0.101799,0.42,0.695
7,AMD,96.86,206,1.25566,1,0.785607,0.9,0.284955,0.2,0.127235,0.64,0.685
8,CDNS,135.32,147,0.929555,0.88,0.455851,0.4,0.336369,0.4,0.174304,0.78,0.615
9,PAYC,442.64,45,0.705389,0.74,0.401371,0.2,0.617612,0.98,0.124278,0.62,0.635


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [158]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True, drop=True)
hqm_dataframe

<ipython-input-158-7ddfc7e27f89>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALGN,537.33,37,0.970887,0.92,1.06399,0.98,0.65407,1,0.208774,0.86,0.94
1,FCX,25.42,786,0.941768,0.9,1.38907,1,0.45281,0.74,0.168387,0.74,0.845
2,TWTR,58.25,343,0.770259,0.8,0.701209,0.84,0.411013,0.68,0.282054,0.98,0.825
3,ALB,141.50,141,1.04082,0.96,0.803159,0.92,0.421946,0.7,0.11116,0.5,0.77
4,XLNX,154.01,129,0.599436,0.64,0.599741,0.74,0.503034,0.88,0.122708,0.56,0.705
5,LRCX,486.89,41,0.683949,0.7,0.559557,0.72,0.599451,0.94,0.101799,0.42,0.695
6,AMD,96.86,206,1.25566,1,0.785607,0.9,0.284955,0.2,0.127235,0.64,0.685
7,PYPL,237.63,84,1.20797,0.98,0.43938,0.32,0.345681,0.44,0.242143,0.94,0.67
8,NTAP,66.51,300,0.0694232,0.1,0.515481,0.62,0.564121,0.92,0.25811,0.96,0.65
9,PAYC,442.64,45,0.705389,0.74,0.401371,0.2,0.617612,0.98,0.124278,0.62,0.635


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [160]:
portfolio_input()

Enter the size of your portfolio:1000000


In [161]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price'])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALGN,537.33,37,0.970887,0.92,1.06399,0.98,0.65407,1,0.208774,0.86,0.94
1,FCX,25.42,786,0.941768,0.9,1.38907,1,0.45281,0.74,0.168387,0.74,0.845
2,TWTR,58.25,343,0.770259,0.8,0.701209,0.84,0.411013,0.68,0.282054,0.98,0.825
3,ALB,141.50,141,1.04082,0.96,0.803159,0.92,0.421946,0.7,0.11116,0.5,0.77
4,XLNX,154.01,129,0.599436,0.64,0.599741,0.74,0.503034,0.88,0.122708,0.56,0.705
5,LRCX,486.89,41,0.683949,0.7,0.559557,0.72,0.599451,0.94,0.101799,0.42,0.695
6,AMD,96.86,206,1.25566,1,0.785607,0.9,0.284955,0.2,0.127235,0.64,0.685
7,PYPL,237.63,84,1.20797,0.98,0.43938,0.32,0.345681,0.44,0.242143,0.94,0.67
8,NTAP,66.51,300,0.0694232,0.1,0.515481,0.62,0.564121,0.92,0.25811,0.96,0.65
9,PAYC,442.64,45,0.705389,0.74,0.401371,0.2,0.617612,0.98,0.124278,0.62,0.635


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [162]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [163]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [164]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 28, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [165]:
writer.save()